In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import time
import logging
from PIL import UnidentifiedImageError

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Custom Dataset class with error handling
class RobustImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
            if self.transform is not None:
                sample = self.transform(sample)
            return sample, target
        except (UnidentifiedImageError, OSError) as e:
            logger.warning(f"Skipping corrupted image {path}: {str(e)}")
            # Try the next image
            return self.__getitem__((index + 1) % len(self))

# Define paths
train_dir = '/content/drive/MyDrive/OIDv4_Dataset/train_split'
test_dir = '/content/drive/MyDrive/OIDv4_Dataset/test_split'

# Enhanced data augmentation for better generalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomPerspective(distortion_scale=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Modified CNN with reduced parameters
class CompressedCNN(nn.Module):
    def __init__(self, num_classes):
        super(CompressedCNN, self).__init__()

        # Initial convolution block with fewer filters
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Reduced from 64 to 32
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        # Deep feature extraction blocks with reduced filters
        self.features = nn.Sequential(
            self._make_block(32, 64),    # Reduced from 64->128 to 32->64
            self._make_block(64, 128),   # Reduced from 128->256 to 64->128
            self._make_block(128, 256),  # Reduced from 256->512 to 128->256
            self._make_block(256, 256)   # Reduced from 512->512 to 256->256
        )

        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Classifier with reduced neurons
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),  # Reduced from 512->1024 to 256->512
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),  # Reduced from 1024->512 to 512->256
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)  # Reduced from 512->num_classes to 256->num_classes
        )

    def _make_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.features(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class MetricsTracker:
    def __init__(self):
        self.train_losses = []
        self.train_accuracies = []
        self.test_losses = []
        self.test_accuracies = []

    def update(self, phase, loss, accuracy):
        if phase == 'train':
            self.train_losses.append(loss)
            self.train_accuracies.append(accuracy)
        else:
            self.test_losses.append(loss)
            self.test_accuracies.append(accuracy)

    def plot_metrics(self):
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(self.train_losses, label='Train Loss')
        plt.plot(self.test_losses, label='Test Loss')
        plt.title('Training and Testing Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.train_accuracies, label='Train Accuracy')
        plt.plot(self.test_accuracies, label='Test Accuracy')
        plt.title('Training and Testing Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()

def train_model(model, dataloaders, num_epochs=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.001,
        epochs=num_epochs,
        steps_per_epoch=len(dataloaders['train'])
    )

    metrics = MetricsTracker()
    best_acc = 0.0
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 30)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            pbar = tqdm(dataloaders[phase], desc=f'{phase.capitalize()} Phase')

            for inputs, labels in pbar:
                try:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                            scheduler.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    total_samples += inputs.size(0)

                    current_loss = running_loss / total_samples
                    current_acc = running_corrects.double() / total_samples
                    pbar.set_postfix({
                        'loss': f'{current_loss:.4f}',
                        'acc': f'{current_acc:.4f}'
                    })
                except Exception as e:
                    logger.error(f"Error in batch processing: {str(e)}")
                    continue

            if total_samples > 0:
                epoch_loss = running_loss / total_samples
                epoch_acc = running_corrects.double() / total_samples

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                metrics.update(phase, epoch_loss, epoch_acc.item())

                if phase == 'test' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    print(f'New best accuracy: {best_acc:.4f}')

                    # Save model with reduced size through quantization
                    quantized_model = torch.quantization.quantize_dynamic(
                        model, {nn.Linear, nn.Conv2d}, dtype=torch.qint8
                    )

                    # Only save the model state dict (not optimizer)
                    torch.save(
                        quantized_model.state_dict(),
                        '/content/drive/MyDrive/90%TrainedModel_compressed.pth'
                    )

                    # Print model size information
                    model_size = os.path.getsize('/content/drive/MyDrive/90%TrainedModel_compressed.pth') / (1024 * 1024)
                    print(f"Model saved. Size: {model_size:.2f} MB")

    time_elapsed = time.time() - start_time
    print(f'\nTraining completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test accuracy: {best_acc:.4f}')

    metrics.plot_metrics()
    return model, metrics

def visualize_predictions(model, dataloader, num_images=6):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    fig = plt.figure(figsize=(15, 10))
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            try:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                for j in range(min(num_images, inputs.size(0))):
                    ax = plt.subplot(2, 3, i * inputs.size(0) + j + 1)
                    ax.axis('off')
                    ax.set_title(f'Predicted: {class_names[preds[j]]}\nTrue: {class_names[labels[j]]}')

                    img = inputs.cpu().data[j].numpy().transpose((1, 2, 0))
                    img = np.clip(img * np.array([0.229, 0.224, 0.225]) +
                                np.array([0.485, 0.456, 0.406]), 0, 1)
                    plt.imshow(img)

                    if i * inputs.size(0) + j + 1 == num_images:
                        plt.show()
                        return
            except Exception as e:
                logger.error(f"Error in visualization: {str(e)}")
                continue

# Function to load the compressed model for inference
def load_compressed_model(model_path, num_classes):
    # Create a model with the same architecture
    model = CompressedCNN(num_classes=num_classes)

    # Create a quantized version of the model
    quantized_model = torch.quantization.quantize_dynamic(
        model, {nn.Linear, nn.Conv2d}, dtype=torch.qint8
    )

    # Load the state dict
    quantized_model.load_state_dict(torch.load(model_path))
    quantized_model.eval()

    return quantized_model

if __name__ == "__main__":
    try:
        # Create datasets with error handling
        train_dataset = RobustImageFolder(train_dir, data_transforms['train'])
        test_dataset = RobustImageFolder(test_dir, data_transforms['test'])

        # Reduce number of workers based on system suggestion
        dataloaders = {
            'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
            'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
        }

        class_names = train_dataset.classes
        num_classes = len(class_names)

        print(f"Classes: {class_names}")
        print(f"Number of training images: {len(train_dataset)}")
        print(f"Number of testing images: {len(test_dataset)}")

        # Create and train model
        model = CompressedCNN(num_classes=num_classes)  # Using the smaller model
        trained_model, training_metrics = train_model(model, dataloaders)

        # Visualize some predictions
        visualize_predictions(trained_model, dataloaders['test'])

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")